In [103]:
def encodelzw(startindex, endindex, rank):
    global DICTIONARY_SIZE
    string = ''
    compressed_data = []
    data = DATA[startindex:endindex+1]
    print('|' * 10, f'HILO {rank} -> CODIFICANDO {data}')
    for symbol in data:
        string_plus_symbol = string + symbol
        if string_plus_symbol in DICTIONARY:
            string = string_plus_symbol
        else:
            compressed_data.append(DICTIONARY[string])
            if len(DICTIONARY) <= MAXIMUM_SIZE:
                DICTIONARY[string_plus_symbol] = DICTIONARY_SIZE
                DICTIONARY_SIZE += 1
            string = symbol
    if string in DICTIONARY:
        compressed_data.append(DICTIONARY[string])
    print('|' * 10, f'RESULTADO DE HILO {rank} -> {compressed_data}')
    RESULT[rank] = compressed_data

In [104]:
def format_compressed_data(compressed_data):
    output_string = ''
    for i in range(len(compressed_data)):
        rank = compressed_data[i]
        for j in range(len(compressed_data[i])):
            code = rank[j]
            if j < len(compressed_data[i]) - 1:
                output_string += f'{code},'
            else:
                output_string += str(code)
        if i < len(compressed_data) - 1:
            output_string += '\n'
    return output_string

In [105]:
# GLOBAL
# LEER ARCHIVO
file = open('data.txt', 'r')
DATA = file.read()
# DICCIONARIO SE INICIALIZA CON 256 CARACTERES ASCII
print('~' * 10, 'INICIALIZANDO DICCIONARIO CON 256 CARACTERES ASCII')
DICTIONARY_SIZE = 256 
DICTIONARY = {chr(i): i for i in range(DICTIONARY_SIZE)}  
# CON 9 BITS SE PUEDE TENER UN DICCIONARIO DE HASTA 512 ESPACIOS
BITS = 9
MAXIMUM_SIZE = pow(2,int(BITS))

# PARALELIZACION
P = 4  # NUMERO DE HILOS UTILIZADOS
offset = len(DATA) // P  # CARACTER EXTRA
RESULT = [[] for p in range(P)]  # LISTA DE RESULTADOS DE CADA HILO (BIDIMENSIONAL)

print('~' * 10, 'CODIFICANDO TROZOS DEL STRING EN CADA HILO')
for rank in range(P):
    # CALCULO DE LOS RANGOS DEL STRING PARA CADA HILO
    startindex = rank * offset
    if rank == P - 1:
        endindex = len(DATA) - 1
    else:
        endindex = startindex + offset - 1
        
    # EJECUCION PARALELA DE LA FUNCION DE CODIFICACION
    # CON CADA TROZO DEL STRING POR HILO
    args = (startindex, endindex, rank)
    t = threading.Thread(target=encodelzw, args=args)
    t.start()
    t.join()

print('=' * 10, f'RESULTADO DE LA CODIFICACION: {RESULT}')
    
print('~' * 10, 'GUARDANDO STRING CODIFICADO')
# MOSTRAR Y GUARDAR RESULTADO
output = open('encoded_data.txt', 'w+')
output_string = format_compressed_data(RESULT)
output.write(output_string)
output.close()

~~~~~~~~~~ INICIALIZANDO DICCIONARIO CON 256 CARACTERES ASCII
~~~~~~~~~~ CODIFICANDO TROZOS DEL STRING EN CADA HILO
|||||||||| HILO 0 -> CODIFICANDO th
|||||||||| RESULTADO DE HILO 0 -> [116, 104]
|||||||||| HILO 1 -> CODIFICANDO th
|||||||||| RESULTADO DE HILO 1 -> [256]
|||||||||| HILO 2 -> CODIFICANDO eg
|||||||||| RESULTADO DE HILO 2 -> [101, 103]
|||||||||| HILO 3 -> CODIFICANDO ame
|||||||||| RESULTADO DE HILO 3 -> [97, 109, 101]
========== RESULTADO DE LA CODIFICACION: [[116, 104], [256], [101, 103], [97, 109, 101]]
~~~~~~~~~~ GUARDANDO STRING CODIFICADO
